Part 1: Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import random as rnd

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Deep learning libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import webbrowser
from json.decoder import JSONDecodeError

# pytest
import pytest
import unittest
from unittest.mock import patch

Part 2: User Authentication
Reference: https://neokt.github.io/projects/audio-music-mood-classification/

In [2]:
# Create App With User Authentication

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="d62cb14d245c47668fd50431a942c046",
client_secret="fc28483fa9a74b22a90d076120fa3df8",
redirect_uri="http://localhost:5000/callback",
scope="user-library-read"))


results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], "-", track['name'])

0 Skryptonite - Polozhenie - Izzamuzzic Remix
1 104 - НЕ ЖАЛЬ (feat. Miyagi & Скриптонит)
2 Truwer - Весь в отца
3 Dequine - Deja Vu
4 Dose - Ветер
5 Автоспорт - Я не буду спать
6 Автоспорт - Пожалуйста, только не домой
7 Автоспорт - Песня о привязанности
8 King Harvest - Dancing in the Moonlight
9 Ashnikko - Daisy
10 Ashnikko - Slumber Party (feat. Princess Nokia)
11 Cuco - Si Me Voy (with The Marías)
12 Hozier - Work Song
13 Jordana - Better in the Dark
14 MUNA - Silk Chiffon
15 WILLIS - I Think I Like When It Rains
16 Ethel Cain - Crush
17 Taylor Swift - seven
18 Lizzy McAlpine - doomsday
19 Bon Iver - For Emma


Part 3: Understanding Spotipy and Querying
References: 
https://neokt.github.io/projects/audio-music-mood-classification/
https://spotipy.readthedocs.io/en/2.22.1/#api-reference

In [3]:
# get categories
categories = sp.categories(limit=50)
for i, cat in enumerate(categories['categories']['items']):
    print(i, cat['name'])

0 Top Lists
1 Hip-Hop
2 Pop
3 Country
4 Latin
5 Rock
6 Summer
7 Workout
8 R&B
9 Dance/Electronic
10 Netflix
11 Indie
12 Mood
13 Sleep
14 Christian & Gospel
15 Regional Mexican
16 Wellness
17 Chill
18 EQUAL
19 Gaming
20 Frequency
21 Kids & Family
22 Party
23 Decades
24 Fresh Finds
25 Jazz
26 Focus
27 Romance
28 Folk & Acoustic
29 K-Pop
30 Instrumental
31 Ambient
32 Alternative
33 In the car
34 Classical
35 Soul
36 Spotify Singles
37 Cooking & Dining
38 Punk
39 Pop culture
40 Blues
41 Desi
42 Arab
43 RADAR
44 Student
45 Anime
46 Tastemakers
47 Afro
48 Comedy
49 Metal


In [4]:
# get playlists from list of categories
cat = categories['categories']['items'][0]
playlists = sp.category_playlists(cat['id'], limit=50)
for i, playlist in enumerate(playlists['playlists']['items']):
    print(i, playlist['name'])

0 Today’s Top Hits
1 RapCaviar
2 Rock This
3 mint
4 Hot Country
5 Viva Latino
6 RNB X
7 Top 50 - USA
8 Top 50 - Global
9 Viral 50 - Global
10 Viral 50 - USA
11 New Music Friday


In [5]:
# get song ids from list of playlist ids
song_ids = []
for i, playlist in enumerate(playlists['playlists']['items']):
    playlist = sp.playlist(playlist['id'])
    for j, item in enumerate(playlist['tracks']['items']):
        song = item['track']
        song_ids.append(song['id'])
        
song_ids[:5]

['7gaA3wERFkFkgivjwbSvkG',
 '4xhsWYTOGcal8zt0J161CU',
 '3rUGC1vUpkDG9CZFHMur1t',
 '2IGMVunIBsBLtEQyoI1Mu7',
 '1BxfuPKGuaTgP7aM0Bbdwr']

In [6]:
# Use song ids to query audio features
features = []
for i in range(0,len(song_ids),50):
    audio_features = sp.audio_features(song_ids[i:i+50])
    for track in audio_features:
        features.append(track)

# Turn the features into a dataframe
features_df = pd.DataFrame(features)

features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.788,0.758,1,-6.513,1,0.0505,0.1980,0.000085,0.1040,0.823,119.008,audio_features,7gaA3wERFkFkgivjwbSvkG,spotify:track:7gaA3wERFkFkgivjwbSvkG,https://api.spotify.com/v1/tracks/7gaA3wERFkFk...,https://api.spotify.com/v1/audio-analysis/7gaA...,214994,4
1,0.943,0.558,2,-4.911,1,0.0568,0.0026,0.000002,0.0937,0.606,104.983,audio_features,4xhsWYTOGcal8zt0J161CU,spotify:track:4xhsWYTOGcal8zt0J161CU,https://api.spotify.com/v1/tracks/4xhsWYTOGcal...,https://api.spotify.com/v1/audio-analysis/4xhs...,138411,4
2,0.750,0.733,6,-3.180,0,0.0319,0.2560,0.000000,0.1140,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1
3,0.868,0.538,5,-8.603,1,0.1740,0.2690,0.000003,0.0901,0.732,99.968,audio_features,2IGMVunIBsBLtEQyoI1Mu7,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,https://api.spotify.com/v1/audio-analysis/2IGM...,231750,4
4,0.552,0.702,9,-5.707,1,0.1570,0.1170,0.000021,0.1050,0.564,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427,4


Part 4: Creating moods and retrieving mood playlists
Moods obtained from: https://github.com/neokt/audio-music-mood-classification/blob/master/05-production.ipynb

In [7]:
# create a list of moods to classify playlists
moods = ['Peaceful', 'Easygoing', 'Tender', 'Romantic', 'Upbeat', 'Empowering', 'Lively', 'Excited', 'Stirring', 'Rowdy', 'Sentimental', 'Sophisticated','Sensual', 'Fiery', 'Energizing','Melancholy', 'Cool', 'Somber', 'Gritty','Yearning', 'Serious','Urgent', 'Defiant', 'Brooding', 'Aggressive']

In [8]:
# get Spotify playlists for each mood. Limit is 50 playlists per mood. For each mood, store the playlist name, id, and mood in a list of dictionaries
playlists = []
for mood in moods:
    mood_playlists = sp.search(q=mood, type='playlist', limit=5)
    for playlist in mood_playlists['playlists']['items']:
        playlists.append({'name': playlist['name'], 'id': playlist['id'], 'mood': mood})

# create a dataframe from the list of dictionaries
playlists_df = pd.DataFrame(playlists)

In [9]:
# get song ids for all the playlists for future querying
song_ids = []
for i, playlist in enumerate(playlists_df['id']):
    playlist = sp.playlist(playlist)
    for j, item in enumerate(playlist['tracks']['items']):
        song = item['track']
        song_ids.append(song['id'])
        
song_ids[:5]

['5hFkGfx038V0LhqI0Uff2J',
 '3RI7rnK1YgmyRcXq9pkMqQ',
 '6fjZ39b5dT9uZzFsVRLwSi',
 '1b1A6Z7pI8myEX3TSyweOk',
 '3xZBHTUT1gLAmDbPz77ETo']

In [29]:
# create a dataframe from the list of dictionaries
playlists_df = pd.DataFrame(playlists)

playlists_df.head()

,name,id,mood
0,Peaceful Piano,37i9dQZF1DX4sWSpwq3LiO,Peaceful
1,Peaceful Meditation,37i9dQZF1DWZqd5JICZI0u,Peaceful
2,slow/peaceful songs,37v5vAWCJu25DR20Fu2jj8,Peaceful
3,Peaceful Music (Instrumental),7MuGX2icynmq9zuklPCbnX,Peaceful
4,Peaceful Retreat,37i9dQZF1DX1T2fEo0ROQ2,Peaceful


In [39]:
# add columns to playlists_df that contain the audio features for the songs
features = []

valid_song_ids = [song_id for song_id in song_ids if song_id is not None]

for i in range(0, len(valid_song_ids), 50):
    audio_features = sp.audio_features(valid_song_ids[i:i + 50])
    valid_audio_features = [track for track in audio_features if track is not None]
    features.extend(valid_audio_features)
    
# features[:5]
        
features_df = pd.DataFrame(features)

# merge the playlists_df and features_df dataframes
df = pd.merge(playlists_df, features_df, left_index=True, right_index=True)

df.head()


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5hFkGfx038V0LhqI0Uff2J,3RI7rnK1YgmyRcXq9pkMqQ,6fjZ39b5dT9uZzFsVRLwSi,1b1A6Z7pI8myEX3TSyweOk,3xZBHTUT1gLAmDbPz77ETo,6yhkANlWR9KWT75aAqXmGm,2bjQvSk5T3A2dmnqqeRhzI,7MDbt6zQQu37YFeVcOpb4U,2ODAJGjI4YZwWp470XsV41,3RO7kRSyGR1WD6vWOI73r9,1ymzdmQYjO3ThdlToYZnTq,5LmaeC0bRQqaHtK781KOhD,6Qqto6duuFLcAUvTEsacwS,3cuvXQjn74rLPC0jSVysUK,4asmLOODbw59npZqVLEvvk,078tv0fBjcReoPb9IL97b7,7A6mWq885WLJUngzwxfQXW,6CWzmoxDJJdIJ5GImw0RHv,6xld6bFu5ST7nickcSK1lP,5JVDIP9IBy6DJB6SVFTX5T,3xdDoLDyvsMgyl1BwVaZ5E,4fv0GOcz0wbPcPR0CjILUg,3bDLjjjT1TNLUI7wnJ8uPW,5tC7qnCutl9HLMAOeUaZxo,5rfMHAaZCA8ym79TN2GdbV,0TsKyZ33N0VP8ZXYgTXPUz,7yQvxdXqotBSRCAYOIOGVt,6SMHndqVp9ZdGmvzQu23ob,3pfjdDVdlQzUjJaw75NvhE,5qowE02tRUQqFf3uTC8tgs,391l8re8aew1rvyNOXUYDD,6GRjlkZ7MxKtRC4jsJzQxv,0jFnlaavbGTER2XNnUztEF,1mKHZDWICROPP1gmOg1mn2,6fHq66kBS2OJiWu3ZGNZj9,6brQG7CUofzdsnNx3qssrh,3IwddX0zjnQ7am17WdP66M,4fG5KccAJKYk1EguGbGKt0,3ANR14gcrMm6asGMNVhivt,6MTI6CKlLOZapPzdT4zJLH,3eGygw8GYE